First, we start by importing modules and reading the dataset to train to a pandas dataframe.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

dataset_train = f'../datasets/dataset_train.csv'
df = pd.read_csv(dataset_train)

# Model training

## Multinomial logistic regression

When there is only 2 possible outcomes for the target, classification is ```binomial```.

```Multinomial``` : The target variable has three or more possible classes.
Our dataset has a discrete number, `4` of possible outcomes = ['Ravenclaw', 'Slytherin', 'Gryffindor', 'Hufflepuff'].

```One-Vs-All Classification``` is a method of multi-class classification.
Breaking down by splitting up the multi-class classification problem into `multiple binary classifier models`.

For a One-vs-All multi-class classification, our dataset, which has k = 4 class labels, we will be using k = 4  ```binary classifiers```.


In [ ]:
df['Hogwarts House'].unique().tolist()


### Binary classifier

For building our model, there will be a set of ```weights```, $W =(𝑤_1,𝑤_2,⋯,𝑤_𝑛)$, that is specific to each ```binary classifier```.

For any given student, characterized by 𝑛 features, the Inputs values are $X =(𝑥_1,𝑥_2,⋯,𝑥_𝑛)$.

The ```dot product``` of ```X``` inputs and ```W``` weights, plus 𝑏 being the bias parameter. will be ```z```.

$$
\mathbf{z} = 𝑏 + 𝑥_1.𝑤_1 + 𝑥_2.𝑤_2 + ... +𝑥_n.𝑤_n
$$

$$
\mathbf{z} = b + \mathbf{X} \cdot \mathbf{W}
$$

with
$$
\mathbf{X} \cdot \mathbf{W} = \sum_{i=1}^n 𝑥_i 𝑤_i
$$

### Sigmoid function

The ```sigmoid function``` also called ```logistic function```  can map input values from a wide range into a limited interval.

$Sigmoid function$
$$ y = g(z) = \frac{1}{1 + e^{-z}} = \frac{e^z}{1 + e^z}$$

This formula represents the `probability of observing the output y = 1`` of a Bernoulli random variable. This variable is either 1 or 0 :
$$
y \in \{0,1\}
$$

The ```sigmoid function``` will transform ```z``` into a value between 0 and 1.
The resulting ```predicted output``` is a probability that a student is meeting the ```binary classifier``` outcome.

$$
y = g(z) = \frac{1}{1 + e^{-z}} = \frac{1}{1 + e^{-(X.W + 𝑏)}}
$$

Since we have 4 possible outcomes,  4 ```binary classifiers``` [(Gryffindor, not Gryffindor), (Ravenclaw, not Ravenclaw), ...],
4 set of weights are needed. 


## Pre-training data processing

- pd.drop() : down to to 10 meaningful features, independent from each other
- pd.dropna() : Dropping rows that contain NaN => down to 1333 rows
- standardize with z-score method

In [ ]:

df.drop(df.columns[2:6], inplace=True, axis = 1)
excluded_features = ["Arithmancy", "Defense Against the Dark Arts", "Care of Magical Creatures"]
df.drop(excluded_features, inplace=True, axis=1)
df.dropna(inplace=True)
df.head()

In [ ]:
def standardize(arr: pd.Series):
    """z-score method, using pandas std"""
    mean = arr.mean()
    std = arr.std()
    return (arr - mean) / std


df_class = df['Hogwarts House'].copy(deep=True)
df_train= df.drop(df.columns[:2], axis = 1)
df_std_train = df_train.agg(lambda feature: standardize(feature))
df_std_train.head()


In [ ]:
df_std_train['Real Output'] = df_class
df_std_train.head()

### Example for One-vs-all :

Actual class y set to 1 or 0 (respectively y or not y)
- 1 = is in house,
- 0 = is in another house

In [ ]:
houses = df['Hogwarts House'].unique()
houses[0]
# gives bool : df['Hogwarts House'] == houses[0]
y_actual = np.where(df['Hogwarts House'] == houses[0], 1, 0)
y_actual

## Model training

for each `classifier` : 
- Input values of 𝑛 features $\mathbf{X}  =(𝑥_1,𝑥_2,⋯,𝑥_𝑛)$ 
- Weights $\mathbf{W} =(𝑤_1,𝑤_2,⋯,𝑤_𝑛)$
- 𝑏, the bias parameter.

so that $$\mathbf{z}= 𝑏 + 𝑥_1.𝑤_1 + 𝑥_2.𝑤_2 + ... +𝑥_n.𝑤_n = 𝑏 + \mathbf{X} \cdot \mathbf{W} = b + \sum_{i=1}^n 𝑥_i 𝑤_i$$,

$\mathbf{z} = b + \mathbf{X} \cdot \mathbf{W} =$ is feeding the logistic function 𝑔, and projects the output as the predicted probability of 𝑦 being equal to 1.

$$
\mathbf{y} = g(\mathbf{z}) = \frac{1}{1 + e^{-\mathbf{z}}} = \frac{1}{1 + e^{-(𝑏 + \mathbf{X} \cdot \mathbf{W})}}
$$

### Input : X Array

a column of ones is added to x_train array so that the bias is multiplied by 1.

$$
\begin{pmatrix}
\ 1 \\
\ x_1 \\
\ 𝑥_2 \\
\ ⋯ \\
\ 𝑥_𝑛 \\
\end{pmatrix}
$$

#### Weights

Transposed matrix of zeros, 
Shape : (one intercept + number of features) x (number of k classifiers)

$$
\begin{pmatrix}
\ b & \ 𝑤_1 & \  𝑤_2 & \ ⋯ & \ 𝑤_𝑛 \\ 
\end{pmatrix}
$$

### Dot product for one output 
$$
\mathbf{z} = 
\begin{pmatrix}
\ 1 \\
\ x_1 \\
\ 𝑥_2 \\
\ ⋯ \\
\ 𝑥_𝑛 \\
\end{pmatrix}
\cdot
\begin{pmatrix}
\ b & \ 𝑤_1 & \  𝑤_2 & \ ⋯ & \ 𝑤_𝑛 \\ 
\end{pmatrix}
=  𝑏 + 𝑥_1.𝑤_1 + 𝑥_2.𝑤_2 + ... +𝑥_n.𝑤_n
$$

### Intializing Model

- Standardized data in a numpy array 
- a column of ones added to the left for the `ìntercept` or bias


In [ ]:
""" Parameters : unstandardized data to train without NaN, output """
df_std_train = df_train.agg(lambda course: standardize(course))
x_train_std = np.array(df_std_train)
ones = np.ones((len(x_train_std), 1), dtype=float)
x_train = np.concatenate((ones, x_train_std), axis=1)
features = df_std_train.columns.tolist()
df_class = df['Hogwarts House'].copy(deep=True)
houses = df_class.unique().tolist()
w_indexes = df_std_train[:-1].columns.insert(0, ['Intercept'])
df_weights = pd.DataFrame(columns=houses, index=w_indexes).fillna(0)
df_weights.head(11)

In [ ]:
df_std_train.head()

# Training one-vs-all

In [ ]:
def sigmoid(arr: np.ndarray) -> np.ndarray:
    return 1 / (1 + np.exp(-arr))

def update_weight_loss(weights, learning_rate, grad_desc):
    return weights - learning_rate * grad_desc

def train_one_vs_all(house, df_class, features, x_train, learning_rate, epochs):
    """
    loss_iter = LogRegTrain.loss_function(y_actual, h_pred)
    gradient = np.dot(x_train.T, (h_pred - y_actual))
    """
    y_actual = np.where(df_class == house, 1, 0)
    weights = np.ones(len(features) + 1).T
    for iter in range(epochs):
        z_output = np.dot(x_train, weights)
        h_pred = sigmoid(z_output)
        tmp = np.dot(x_train.T, (h_pred - y_actual))
        grad_desc = tmp / y_actual.shape[0]
        weights = update_weight_loss(weights, learning_rate, grad_desc)
    return weights


learning_rate = 0.1
epochs = 1000
for house in houses:
    weights = train_one_vs_all(house, df_class, features, x_train, learning_rate, epochs)
    df_weights[house] = weights
print("alpha = ", learning_rate, "  iterations =", epochs)
df_weights.head(11)

    We have the parameters (biases and weights) for our logistic regression model !